![](https://i.pinimg.com/564x/38/aa/d5/38aad51d8f9e7b4d58b37d2600aa9c52.jpg)


Introduction:
Netflix was founded in 1997, the headquarter is in California, it is a subscription-based streaming audio and video platform that people can watch TV shows and movies. Netflix is now a very popular service in the world, the global subscriptions have exceeded 200 million people in 2020. 
The reason why Netflix become so hot :
1. High output rate: you won't be nervous or anxious if there is no movie/TV show to watch, because there are a dozens of series on the platform from 1930s to 2021.
2. Netflix Origin Series: most of the subscription-based platform are most afraid of consumers unsubscribing, however Netflix launch all kinds of original albums which has great quality, that people love it so much!
3. Covid-19: Due to the epidemic, people loss the opportunity to go outside and have fun, so they change the entertainment to watch Netflix at home:) 

Below is the visualization and analysis base on the Netflix database.  

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pandas import DataFrame
import missingno as msno
from matplotlib.colors import ListedColormap
import altair as alt

%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')
df

In [ ]:
#視覺化表示缺失值
#df.isnull().sum()
msno.bar(df)


↑ There are totally 7787 data in the dataset. We can find that there are a lot of missimg values in director, and cast columns.

In [ ]:
#資料中movie和TV的比例
Type = df.groupby(by='type').show_id.count()
Type =pd.DataFrame(Type)
#Type
colors = ['#ff4e50','#fcbe32']
separated = (.1,0)
plt.figure(figsize=(12, 6))
plt.title('Propotion of the Data')
plt.pie(Type.show_id, labels = Type.index ,autopct='%1.1f%%', explode=separated,colors = colors)
plt.show()

In [ ]:
Rating = df.groupby(by='rating').show_id.count()
Rating =pd.DataFrame(Rating)
Rating =Rating.reset_index()
plt.figure(figsize=(12, 6))
plt.title('Netflix Rating')
plt.xlabel('Rating')
plt.ylabel('Number of Issues')
plt.bar(Rating.rating, Rating.show_id , color='#ff4e50')
plt.xticks(rotation=90)
plt.show()

# We found that there are 7787 movies/ TV show in the dataset, and we are wondering that if the shows were filmed by the same director?

In [ ]:
#同個導演拍過幾部電影/電視劇在netflix上架
a = df.groupby(by='director').show_id.count()
a= pd.DataFrame(a)
a.rename(columns={'show_id':'Count'}, inplace=True)
a = a.sort_values(by='Count',ascending=False)
a = a.reset_index()
#plt.bar(a.index, a.show_id)
#在Netflix只有上架一部作品的有3312個導演，但也有1個導演一個人就在Netflix上架了18部作品
netflix_director_count = a.groupby(by='Count').director.count()
netflix_director_count = pd.DataFrame(netflix_director_count)
netflix_director_count = netflix_director_count.reset_index()
netflix_director_count
# plt.bar(netflix_director_count.Count, netflix_director_count.director)
# plt.xlim((0, 18))
# plt.xlabel("Number of Movies/ TV in Netflix",fontsize=13)
# plt.ylabel("How many  Movies/ TV",fontsize=13)




In [ ]:
a = df.groupby(by='director').show_id.count()
a= pd.DataFrame(a)
a.rename(columns={'show_id':'Count'}, inplace=True)
a = a.sort_values(by='Count',ascending=False)
a = a.reset_index()
b = a.head(10)
b
plt.figure(figsize=(12, 6))

plt.plot(b.director, b.Count)
plt.title('Director with the most works')
plt.xlabel('Director')
plt.ylabel('Numbers of  works')
plt.xticks(rotation=90)
plt.show()

↑ Most of the directors film 1 movie/ TV show on Netflix.
But there is 1 director who film 18 shows on Netflix.


# That's look at director Raúl Campos, Jan Suter's Portfolio 

In [ ]:
df[df['director']=='Raúl Campos, Jan Suter']

↑ We found that Raúl Campos, Jan Suter is a movie director, and all of his movies are produced in South America, they are all Standup Comedy.

In [ ]:
df[df['director']=='Marcus Raboy']

↑Marcus Raboy not only make movies, he is also a TV show director, most of them are stand-up comedy. 

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(x='rating',data = df,hue='type',palette='YlOrBr')
plt.xticks(rotation = 90)
plt.show()

↑ Most of the series on Netflix is suitable for adults, some content is not suitable for children younger than 17 years old

In [ ]:
#top 15 countries (Use sunburst)
import plotly.express as px
top_country = df["country"].value_counts()[:15]
top_country = pd.DataFrame(top_country)
top_country = top_country.reset_index()
fig =px.sunburst(top_country,  path=['index'],values='country',color_continuous_scale='OrRd'
                ,color='country')
fig.show()

In [ ]:
release_year = df.groupby(['release_year','type']).show_id.count()
release_year = pd.DataFrame(release_year)
release_year = release_year.reset_index()
release_year[release_year.release_year<2000].show_id.sum()
#2000年以前的片共有449部，此份資料庫共有7789部片
latest = release_year[release_year.release_year>2000]
# year = sns.load_dataset("latest")

plt.figure(figsize=(12, 6))
sns.barplot(x='release_year',y='show_id',hue='type',data=latest, palette='YlOrBr')
plt.title('Number of Netflix series released in recent years')

↑ Most of the series were released after 2015.
There's an interesting discovery: Before 2019, the number of movies released each year is much larger than that of TV series.

In [ ]:
#seperate the fata in column: date_added
df['add_month'] = df.date_added.str.split(' ').str.get(0)
df['add_year'] = df.date_added.str.split(',').str.get(1)

#Use Heatmap to see when Movies/TV Shows release over years
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'][::-1]
#unstack：將資料的行“旋轉”為列
df5 = df.groupby('add_year')['add_month'].value_counts().unstack().fillna(0)[month_order].T
#df5
plt.figure(figsize=(12, 6))
sns.heatmap(df5,linewidths = 0.05, cmap="OrRd")
plt.title('Movies/ TV shows Added time')


In [ ]:
#colors = ''
usa_recent5 =  df[df.release_year >= 2016]
g = sns.catplot(
    data=usa_recent5, kind="count",
    x="release_year", hue="type",
    ci="sd", palette='YlOrBr')
ax = plt.gca()
# ylim max value to be set
y_max = usa_recent5['type'].value_counts().max() 
# Iterate through the list of axes' patches
#數值
# for p in ax.patches:
#     ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
#             fontsize=12, color='black', ha='center', va='bottom')
plt.title('Last 5 years trends in Netflix by USA',size='15')
plt.show()

# Curious about the genres in Netflix, 
# What are the movies/ TV Shows released on Netflix?

In [ ]:
#the data in column: listed_in included a lot of categories, so we need to forst clean the data, and make the right category.
df2 = df.drop('listed_in', axis=1).join(df['listed_in'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True).rename('Category'))
Category_cut = df2.groupby(by = 'Category').show_id.count()
Category_cut = pd.DataFrame(Category_cut)
Category_cut.sort_values(by='show_id', ascending= False)
df2
#df2 #將Category各項拆開
# df2["Category"].value_counts()[:10].plot(kind="barh", color='#E53A40')
# plt.title("Top 10 genres")

########
import squarify
top_genres = df2["Category"].value_counts()[:10]
top_genres
#plt.rc('font', size=10)
plt.figure(figsize=(12, 6))
squarify.plot(sizes=top_genres.values, label=top_genres.index, linewidth=4,
              color=sns.color_palette("RdGy", n_colors=20) )
plt.axis('off')
plt.show()

I use Tree Map to present the Genres on Netflix. we can find that 
**International Movies > Dramas > Comedies**
 
 
There is a small detail that needs special attention：
When reading the final charts, **make sure that there are no repeated data**.

When grouping and cleaning the data, i found that if we split data by ',' there will be two Drama
one with spaces in front of the word, the other don't have spaces, so python will view it as different data but we actually need to combine the data.

In [ ]:
df2[df2["type"]=="TV Show"]["Category"].value_counts()[:10].plot(kind="barh", color='#E53A40')
plt.title("Top 10 TV Show genres")

In [ ]:
df2[df2["type"]=="Movie"]["Category"].value_counts()[:10].plot(kind="barh", color='#E53A40')
plt.title("Top 10 movie genres")

↑ The hottest genres are different between movies and TV shows!

In [ ]:
tag = "International Movies" #Kids TV shows on Netflix
df["relevant"] = df['listed_in'].fillna("").apply(lambda x : 1 if tag.lower() in x.lower() else 0)
com = df[df["relevant"] == 1]
com[["title", "country","release_year"]].head(10)

↑ After understanding most of the content of Netflix, if we want to find movies in a specific category, we can use the above methods.
It's a pity that we don't have the ranking or customer views number of the movies, or we might get the most popular series and find one that we are interested and enjoy it!